In [ ]:
!pip install biopython gensim umap-learn scikit-learn==1.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


# Create W2V Model - Rice

4-mers / Vector: 64

In [1]:
kmer_size = 4
vector_size = 64

In [2]:
from Bio import SeqIO
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from joblib import load

def circrna_to_kmers(circrna_sequence, k=kmer_size):
  kmers = []
  for i in range(0, len(circrna_sequence)-k+1):
    kmer = circrna_sequence[i:i+k]
    kmers.append(kmer)
  return kmers

fasta_handle = open('osaj43883_genomic_seq.txt', 'r')
fasta_parser = SeqIO.parse(fasta_handle, 'fasta')

with open('oryza_corpus.txt', 'w') as corpus_handle:
  for record in fasta_parser:
    record_kmers = circrna_to_kmers(str(record.seq))
    corpus_handle.write(' '.join(record_kmers) + '\n')

w2v_model = Word2Vec(vector_size=vector_size)
w2v_model.build_vocab(corpus_file='oryza_corpus.txt')

w2v_model.corpus_count

# Training the w2v model
w2v_model.train(corpus_file='oryza_corpus.txt', total_words=w2v_model.corpus_total_words, epochs=1)

# Save the model
w2v_model.save("word2vec_model_rice_4mer_64.bin")

# Create W2V Model - Maize

4-mers / Vector: 64

In [3]:
fasta_handle = open('zma10381_genomic_seq.txt', 'r')
fasta_parser = SeqIO.parse(fasta_handle, 'fasta')

with open('maize_corpus.txt', 'w') as corpus_handle:
  for record in fasta_parser:
    record_kmers = circrna_to_kmers(str(record.seq))
    corpus_handle.write(' '.join(record_kmers) + '\n')

w2v_model = Word2Vec(vector_size=vector_size)
w2v_model.build_vocab(corpus_file='maize_corpus.txt')

w2v_model.corpus_count

# Training the w2v model
w2v_model.train(corpus_file='maize_corpus.txt', total_words=w2v_model.corpus_total_words, epochs=1)

# Save the model
w2v_model.save("word2vec_model_maize_4mer_64.bin")

# Create rice dataset W2Vec
4mer / vector: 64

In [4]:
def circrna_to_kmers(circrna_sequence, k=kmer_size):
  kmers = []
  for i in range(0, len(circrna_sequence)-k+1):
    kmer = circrna_sequence[i:i+k]
    kmers.append(kmer)
  return kmers

In [5]:
def circrna_to_vec(circrna_sequence, k=kmer_size):
  vec = np.zeros(vector_size)
  kmers = circrna_to_kmers(circrna_sequence, k=k)
  for kmer in kmers:
    if kmer in w2v_model.wv:
      vec = vec + w2v_model.wv[kmer]
  return vec

In [6]:
w2v_model = Word2Vec.load("word2vec_model_rice_4mer_64.bin")

In [7]:
circ_df = pd.read_excel('rice_db.xlsx')

In [8]:
list = circ_df['seq']

columns = [f'wc_4mer_{v+1}' for v in range(vector_size)]

df_vecs = pd.DataFrame(columns=columns)

for record in list:
  df_vecs = df_vecs._append(
      [
       dict(zip(columns, circrna_to_vec(record)))
       ],
      ignore_index=True)

df_vecs

<ipython-input-8-1cc24255fe7b>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_vecs = df_vecs._append(


,wc_4mer_1,wc_4mer_2,wc_4mer_3,wc_4mer_4,wc_4mer_5,wc_4mer_6,wc_4mer_7,wc_4mer_8,wc_4mer_9,wc_4mer_10,...,wc_4mer_55,wc_4mer_56,wc_4mer_57,wc_4mer_58,wc_4mer_59,wc_4mer_60,wc_4mer_61,wc_4mer_62,wc_4mer_63,wc_4mer_64
0,-0.883167,-27.182693,20.293533,-25.891542,-0.327645,-43.876714,-35.364133,7.127558,10.932993,29.119079,...,-11.918541,12.279123,9.011936,24.100466,24.308761,8.260166,-15.024550,6.002867,1.507978,11.182904
1,105.458237,-205.173828,-3.106734,-66.383620,-86.532058,-86.942771,-16.478562,69.381458,-169.681112,138.720767,...,-202.630082,93.116379,44.281993,-0.619377,152.964642,-35.490563,-43.889002,-91.200162,-124.374574,-16.868270
2,-1.159947,-71.042494,77.322914,-94.434832,-5.155498,-70.740742,-40.509667,9.325278,70.970050,-3.671634,...,79.834526,58.315066,76.990689,112.299793,-20.830383,1.719916,-88.444141,-24.392441,45.363948,-59.348649
3,47.428846,-140.967152,49.172377,-149.280351,-61.012022,-123.681630,71.223812,43.296764,-3.876328,133.883051,...,-110.379918,95.236562,-35.961229,-53.723221,140.296401,-42.802093,-76.339258,-90.809471,35.002862,-28.652204
4,-15.689717,93.998905,-60.225610,-8.531456,90.296971,-83.342529,-67.898767,10.876628,131.815458,-69.384569,...,46.177289,-30.089921,-7.881961,150.701610,-55.671148,27.387335,-40.329298,102.078460,-2.521554,-52.439878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63043,31.380042,-38.655341,-28.463270,-57.076541,2.506140,-83.795310,46.455955,9.556831,11.938837,68.838223,...,-99.780442,11.119664,-42.563230,6.812708,109.639464,18.371555,15.229715,19.704079,18.512268,-7.923207
63044,-52.817674,173.723084,-168.865193,-191.782014,225.340244,-136.896555,99.299521,110.302795,476.589036,-10.621375,...,-28.747048,-266.892655,-226.219896,148.678723,169.298494,154.680326,-9.554081,189.871215,151.594287,-113.771354
63045,-44.861677,118.482361,-155.060860,-165.348231,157.115416,-120.635489,34.688737,95.356694,355.038870,7.860272,...,14.278233,-158.246766,-151.757697,157.042645,66.899736,77.941490,-25.593593,139.023977,163.132161,-55.952635
63046,21.290343,-23.974379,-26.970684,-11.620159,-1.023653,12.916220,39.783063,37.515830,-14.936447,32.922216,...,-34.013577,-18.169211,-27.822827,-12.233814,51.532819,-0.529952,14.858692,-29.367595,-6.680204,-17.777768


In [9]:
new_df = pd.concat([circ_df, df_vecs], axis=1)

new_df.head()

,circName,circID,gene,isoform,stress,tissue,chr,start,end,strand,...,wc_4mer_55,wc_4mer_56,wc_4mer_57,wc_4mer_58,wc_4mer_59,wc_4mer_60,wc_4mer_61,wc_4mer_62,wc_4mer_63,wc_4mer_64
0,osa-circ1-OS01T0723400,1:30167620-30167771_+,OS01T0723400,OS01T0723400-01,-,multipleTissue,1,30167620,30167771,+,...,-11.918541,12.279123,9.011936,24.100466,24.308761,8.260166,-15.024550,6.002867,1.507978,11.182904
1,osa-circ2-OS03T0223400,3:6461672-6462146_-,OS03T0223400,OS03T0223400-01,-,multipleTissue,3,6461672,6462146,-,...,-202.630082,93.116379,44.281993,-0.619377,152.964642,-35.490563,-43.889002,-91.200162,-124.374574,-16.868270
2,osa-circ3-OS11T0210300,11:5715883-5716030_-,OS11T0210300,OS11T0210300-02,-,multipleTissue,11,5715883,5716030,-,...,79.834526,58.315066,76.990689,112.299793,-20.830383,1.719916,-88.444141,-24.392441,45.363948,-59.348649
3,osa-circ4-OS02T0200900,2:5631023-5631244_-,OS02T0200900,OS02T0200900-02,-,multipleTissue,2,5631023,5631244,-,...,-110.379918,95.236562,-35.961229,-53.723221,140.296401,-42.802093,-76.339258,-90.809471,35.002862,-28.652204
4,osa-circ5-OS05T0494800,5:24302336-24302448_+,OS05T0494800,OS05T0494800-01,-,multipleTissue,5,24302336,24302448,+,...,46.177289,-30.089921,-7.881961,150.701610,-55.671148,27.387335,-40.329298,102.078460,-2.521554,-52.439878


In [10]:
new_df.to_excel('rice_w2vec_4mer_64_dataset.xlsx', index=False)

# Create maize dataset W2Vec
4mer / vector: 64

In [11]:
w2v_model = Word2Vec.load("word2vec_model_maize_4mer_64.bin")

In [12]:
circ_df = pd.read_excel('maize_db.xlsx')

In [13]:
list = circ_df['seq']

columns = [f'wc_4mer_{v+1}' for v in range(vector_size)]

df_vecs = pd.DataFrame(columns=columns)

for record in list:
  df_vecs = df_vecs._append(
      [
       dict(zip(columns, circrna_to_vec(record)))
       ],
      ignore_index=True)

df_vecs

<ipython-input-13-1cc24255fe7b>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_vecs = df_vecs._append(


,wc_4mer_1,wc_4mer_2,wc_4mer_3,wc_4mer_4,wc_4mer_5,wc_4mer_6,wc_4mer_7,wc_4mer_8,wc_4mer_9,wc_4mer_10,...,wc_4mer_55,wc_4mer_56,wc_4mer_57,wc_4mer_58,wc_4mer_59,wc_4mer_60,wc_4mer_61,wc_4mer_62,wc_4mer_63,wc_4mer_64
0,1.054163,-70.318502,31.744584,-64.096288,-1.076926,-10.856176,-37.548611,-23.703388,-75.991560,23.933067,...,77.393580,74.043442,31.328972,-42.454553,-9.889269,12.356363,25.170455,-63.234160,14.726255,45.444464
1,174.767270,-73.903631,124.741698,-5.222348,-9.271994,-4.809022,309.751277,609.374171,115.567131,-441.566547,...,-246.910638,-124.831894,20.303335,274.302632,170.436522,-168.772085,199.075725,89.297070,236.255482,-249.192678
2,50.894189,-58.097239,28.014851,-100.480574,-33.667196,65.220866,-15.569865,45.853734,-56.197802,57.584769,...,111.011179,54.774788,6.009535,20.913969,43.223858,23.635217,-49.815108,-38.091709,-61.853444,44.964491
3,-7.524265,-27.744542,25.204052,-46.512765,-1.816013,-10.219601,-47.531095,-12.055163,-59.132948,17.645127,...,69.846905,45.887955,40.177923,-33.980506,12.350419,2.915954,34.710866,-23.027774,48.889823,39.538235
4,-33.796090,-77.314681,-63.401630,9.785242,-15.727631,-29.428019,69.532630,9.851196,-48.430253,71.579849,...,-134.859190,8.899827,48.499065,33.213476,-98.970419,-21.683833,-60.300901,79.830985,13.563471,-57.202161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38780,49.595283,-48.926864,12.556854,-66.998083,-61.329110,104.545447,-38.359329,2.799949,-28.250893,58.056350,...,79.669001,-1.329121,-11.341707,34.766446,32.359054,40.640692,-80.783545,-24.384338,-134.665387,43.139605
38781,14.027877,-28.164322,-19.495092,7.535586,-13.361632,21.706241,30.512707,41.917706,-11.427917,18.801071,...,-48.971628,-5.837088,6.745489,20.549800,-21.287249,7.270882,-46.582860,9.137553,-46.465598,7.007287
38782,9.829941,-25.370295,58.773672,-65.068565,-108.632346,168.078792,-49.889432,-59.258827,-78.610244,128.696234,...,41.467277,19.271900,-13.024908,12.326810,60.847509,48.558913,-131.029256,10.569566,-302.425457,73.157865
38783,289.828549,-85.827741,28.814229,-86.503917,-118.521286,283.465877,182.127694,600.392828,73.770635,-95.601840,...,36.850450,11.178022,-96.514191,299.437490,246.190646,146.167286,-1.062652,-41.270287,-290.228417,-86.383874


In [14]:
new_df = pd.concat([circ_df, df_vecs], axis=1)

new_df.head()

,circName,circID,gene,isoform,stress,tissue,chr,start,end,strand,...,wc_4mer_55,wc_4mer_56,wc_4mer_57,wc_4mer_58,wc_4mer_59,wc_4mer_60,wc_4mer_61,wc_4mer_62,wc_4mer_63,wc_4mer_64
0,zma-circ1-Zm00001d002325,2:10317309-10317467_-,Zm00001d002325,Zm00001d002325_T001,-,multipleTissue,2,10317309,10317467,-,...,77.393580,74.043442,31.328972,-42.454553,-9.889269,12.356363,25.170455,-63.234160,14.726255,45.444464
1,zma-circ2-Zm00001d038675,6:162376852-162378246_+,Zm00001d038675,Zm00001d038675_T004,-,multipleTissue,6,162376852,162378246,+,...,-246.910638,-124.831894,20.303335,274.302632,170.436522,-168.772085,199.075725,89.297070,236.255482,-249.192678
2,zma-circ3-Zm00001d038163,6:150032431-150032595_+,Zm00001d038163,Zm00001d038163_T001,-,multipleTissue,6,150032431,150032595,+,...,111.011179,54.774788,6.009535,20.913969,43.223858,23.635217,-49.815108,-38.091709,-61.853444,44.964491
3,zma-circ4-Zm00001d049552,4:34381638-34381747_-,Zm00001d049552,Zm00001d049552_T004,-,multipleTissue,4,34381638,34381747,-,...,69.846905,45.887955,40.177923,-33.980506,12.350419,2.915954,34.710866,-23.027774,48.889823,39.538235
4,zma-circ5-Zm00001d032567,1:230724608-230725226_-,Zm00001d032567,Zm00001d032567_T001,-,multipleTissue,1,230724608,230725226,-,...,-134.859190,8.899827,48.499065,33.213476,-98.970419,-21.683833,-60.300901,79.830985,13.563471,-57.202161


In [15]:
new_df.to_excel('maize_w2vec_4mer_64_dataset.xlsx', index=False)